# Importing Libraries

In [1]:
import tensorflow as tf
import pathlib
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import IPython.display as display
from PIL import Image
import os, shutil
import numpy as np
import matplotlib.pyplot as plt

# Loading Dataset

from google.colab import drive
drive.mount('/content/drive')

# Set the correct dataset path
dataset_path = 'test_train_valid'

# Verify the dataset path
import os
print(os.listdir(dataset_path))

In [ ]:
img_size = (244, 244)
batch_size = 32
train_dir= 'test_train_valid/train'
val_dir= 'test_train_valid/validation'
test_dir= 'test_train_valid/test'

# Data Preprocessing and Augmentation

In [ ]:
import tensorflow as tf
# Training set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=img_size,
  batch_size=batch_size,
  #color_mode='grayscale',
  label_mode = 'categorical')

# Validation set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_dir,
  seed=123,
  image_size=img_size,
  batch_size=batch_size,
  #color_mode='grayscale',
  label_mode = 'categorical')

# Test set
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=img_size,
  batch_size=batch_size,
  #color_mode='grayscale',
  label_mode = 'categorical')

In [ ]:
def norm_data (ds):
  prepr_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
  norm_ds = ds.map(lambda x, y: (prepr_layer(x), y))

  return norm_ds

In [ ]:
def augment_data1 (ds, crop_size):

  layer1 = tf.keras.layers.experimental.preprocessing.CenterCrop(crop_size, crop_size)
  aug_ds = ds.map(lambda x, y: (layer1(x), y))

  layer2 = tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.05, 0.05) )
  aug_ds = aug_ds.map(lambda x, y: (layer2(x), y))

  layer3 = tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(-0.1, -0.01), width_factor=(-0.1,  -0.01))
  aug_ds = aug_ds.map(lambda x, y: (layer3(x), y))

  return aug_ds


In [ ]:
img_height = 244
train_ds_aug = augment_data1 (train_ds, img_height-20)
train_ds_norm = norm_data (train_ds_aug)
val_ds_aug = augment_data1 (val_ds, img_height-20)
val_ds_norm = norm_data (val_ds_aug)
AUTOTUNE = tf.data.AUTOTUNE

train_ds_norm = train_ds_norm.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_norm = val_ds_norm.cache().prefetch(buffer_size=AUTOTUNE)
AUTOTUNE = tf.data.AUTOTUNE

train_ds_aug = train_ds_aug.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_aug = val_ds_aug.cache().prefetch(buffer_size=AUTOTUNE)

test_ds_aug = augment_data1 (test_ds, img_height-20)
test_ds_norm = norm_data (test_ds_aug)

AUTOTUNE = tf.data.AUTOTUNE
test_ds_norm = test_ds_norm.cache().prefetch(buffer_size=AUTOTUNE)

# Ensemble Architecture

In [ ]:
import tensorflow as tf
import pathlib
import numpy as np
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory

checkpoint_path = "Ensemble_Checkpoint/Ensemble_Checkpoint.h5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_loss', mode='min', verbose=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True, verbose=1)

# Load pre-trained models
model_CNN = load_model('/content/drive/My Drive/Kvasir Code/CNN.h5')
#model_dnsnt201 = load_model('/content/drive/My Drive/Kvasir Code/dnsnt201.h5')
#model_rsnet152 = load_model('/content/drive/My Drive/Kvasir Code/rsnt152.h5')
model_rsnt101 = load_model('/content/drive/My Drive/Kvasir Code/rsnt101.h5')

# Remove top layers to get feature extractors
model_CNN_feature = tf.keras.Model(inputs=model_CNN.input, outputs=model_CNN.layers[-2].output)
#model_dnsnt201_feature = tf.keras.Model(inputs=model_dnsnt201.input, outputs=model_dnsnt201.layers[-2].output)
#model_rsnet152_feature = tf.keras.Model(inputs=model_rsnet152.input, outputs=model_rsnet152.layers[-2].output)
model_rsnt101_feature = tf.keras.Model(inputs=model_rsnt101.input, outputs=model_rsnt101.layers[-2].output)

# Input layer
num_classes = 8
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

# Extract features
CNN_features = model_CNN_feature(inputs)
#dnsnt201_features = model_dnsnt201_feature(inputs)
#rsnet152_features = model_rsnet152_feature(inputs)
rsnt101_features = model_rsnt101_feature(inputs)

# Concatenate features
combined_features = Concatenate()([CNN_features, rsnt101_features]) #, dnsnt201_features, rsnet152_features])

# Add new dense layers on top
x = Dense(512, activation='relu')(combined_features)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

# Create the new ensemble model
ensemble_model = Model(inputs=inputs, outputs=outputs)

# Compile and train the ensemble model
opt1 = optimizers.Adam(learning_rate=2e-5)
ensemble_model.compile(optimizer=opt1, loss="categorical_crossentropy", metrics=['accuracy'])

history_ensemble = ensemble_model.fit(
    train_ds_norm,
    epochs=1,
    validation_data=val_ds_norm,
    callbacks=[checkpoint_callback, early_stopping_callback]
)


In [ ]:
# Plotting the Loss and Accuracy Curves
plt.figure(figsize=(12, 4))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history_ensemble.history['loss'], label='train_loss')
plt.plot(history_ensemble.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_ensemble.history['accuracy'], label='train_accuracy')
plt.plot(history_ensemble.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Curve')
plt.legend()

plt.show()

# Evaluate the model
loss, accuracy = ensemble_model.evaluate(val_ds_norm)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Generate predictions
y_pred_prob = ensemble_model.predict(val_ds_norm)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Get true labels
y_true = []
for images, labels in val_ds_norm:
    y_true.extend(tf.argmax(labels, axis=1).numpy())

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=val_ds_norm.class_names))

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=val_ds_norm.class_names, yticklabels=val_ds_norm.class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Calculate accuracy, precision, recall, F1-score, AUC-ROC
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
roc_auc = roc_auc_score(tf.one_hot(y_true, depth=num_classes), y_pred_prob, average='weighted', multi_class='ovr')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"AUC-ROC: {roc_auc}")
